<a href="https://colab.research.google.com/github/slnarayan/AddressEvaluationLLM/blob/main/AddressEvalUsingLLMEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 8.8 MB/s e

In [2]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

llm_model = "gpt-3.5-turbo-0301"

def get_completion(prompt, model=llm_model):
  client = OpenAI( api_key = os.environ['OPENAI_API_KEY'],)
  messages = [{"role": "user", "content":prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.0,
  )
  return response.choices[0].message.content

get_completion("What is 1+1?")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.prompts import ChatPromptTemplate
import pandas as pd
import numpy as np

chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [10]:
#Load the excel file
file = 'address_org_std.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

#df = pd.read_csv('address_org_std.csv')

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [16]:
!pip install "langchain[docarray]"
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287617 sha256=bfa0afe39717d06e5e6bb48b5ede82de621c08db795ced77a73a49ed2fc9b5a1
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [47]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [ ]:
query1 = "10 Florence St, Malden, MA"
query2 = "959 Howe avenue, Shelton, CT"
query3 = "1018 Oleander Lane"

In [ ]:
resp = db.similarity_search_with_score(query3,k=2)
resp

[(Document(page_content='\ufeffaddress_std: 1018 Oleander Ln, Waxhaw, NC, 28173', metadata={'source': 'address_std.csv', 'row': 0}),
  0.8707266034609397),
 (Document(page_content='\ufeffaddress_std: 10 Florence St, #201, Malden, MA 02148', metadata={'source': 'address_std.csv', 'row': 1}),
  0.7761470260806635)]